In [1]:
#default_exp modules.pointpillars


# Pointpillars wrapper module
> This is a wrapper for all of the separate pointpillar modules. This includes the featurenet, the backbone,
> the detection head and also the box matching module.

In [2]:
import sys
sys.path.append("/home/qhs67/git/bachelorthesis_sven_thaele/code/")

In [3]:
#export
import logging
import torch
import torch.nn as nn
import torch.nn.functional as F

from pointpillars.modules.featurenet import FeatureNet
from pointpillars.modules.backbone import Backbone
from pointpillars.modules.head import Head
from pointpillars.modules.boxmatch import BoxMatch

logger = logging.getLogger(__name__)

NameError: name 'nn' is not defined

In [ ]:
#export
class PointPillars(nn.Module):

    def __init__(self, conf, training: bool = True, use_boxmatch: bool = False):
        """A wrapper which combines all of the individual pointpillar modules.

        :param conf:
        :param training: If used in training or prediction mode.

        :returns: Depending on training or prediction mode
        """
        super(PointPillars, self).__init__()

        self.conf = conf
        self.training = training
        self.use_boxmatch = use_boxmatch

        self.featurenet = FeatureNet(conf['featurenet'], conf['pillars'])
        self.backbone = Backbone(conf['featurenet'])
        self.head = Head(conf['head'])
        self.boxmatch = BoxMatch(conf['pillars'], conf['head'], training)

    def __call__(self,
                 batch: torch.Tensor,
                 ind_batch: torch.Tensor,
                 label_batch: torch.Tensor = None,
                 label_mask: torch.Tensor = None):
        """
        Passes the given parameters forward through the network. Can either be for training or at interference
        time. The pil_batch contains the pillar points and the ind_batch contains the pillar values.
        When in training mode, label_batch is expected containing the labels in a tensor form.

        :param batch:     Tensor(nb_attributes, nbr_pillars w. zero_padding, nb_points_in_pillars)
                          Tensor containing the points sorted into their corresponding pillar
        :param ind_batch: Tensor(nb_pillars, 2)
                          Tensor containing the unique index for each pillar to later identify the position
        :param label_batch: Tensor(batch_size, nb_labels, label_attributes)
                            Only required if in training mode.

        :returns:
        """
        batch = self.featurenet(batch, ind_batch)
        batch = self.backbone(batch)
        batch = self.head(batch)
        batch = self.boxmatch(batch, label_batch, label_mask)

        if not self.training:
            occ_threshold = 0.9995
            pred_occ, pred_cls, pred_head, pred_box = batch

            pred_occ = torch.sigmoid(pred_occ)
            pred_cls = torch.sigmoid(pred_cls)
            pred_head = torch.sigmoid(pred_cls)

            if self.use_boxmatch:
                raise NotImplementedError("Option using Boxmatch not yet implemented.")

            # apply threshold for matching
            else:
                # find occ values greater than threshold
                # pred_occ (bs_size, nb_boxes)
                cond = pred_occ[0] >= occ_threshold
                indices = torch.nonzero(cond, as_tuple=False).squeeze(1)

                pred_anchors = torch.index_select(pred_box[0], 0, indices)

                return pred_anchors, label_mask



        return batch



In [ ]:
#export
def init_weights(m):
    torch.nn.init.uniform(m.weight)